In [1]:
import os
import numpy as np
import pandas as pd
import time
import sys
sys.path.append('../')
import progressbar as P
from helper import getLLPs,getModelDict,splitModels
from ATLAS_data.effFunctions import (getMuonRecoEff,getTriggerEff,getTrackEff,
                                     getSelectionEff,getTargetMass,getMassSelEff,
                                     massLong,massShort)
from atlas_susy_2018_42_Recast import getHSCPCandidates,applyHSCPSelection,applyIsolation,applyMuonTagging,removeFromMET,applyMTCut

delphesDir = os.path.abspath("../DelphesLLP")
os.environ['ROOT_INCLUDE_PATH'] = os.path.join(delphesDir,"external")

import ROOT
import xml.etree.ElementTree as ET


ROOT.gSystem.Load(os.path.join(delphesDir,"libDelphes.so"))

ROOT.gInterpreter.Declare('#include "classes/SortableObject.h"')
ROOT.gInterpreter.Declare('#include "classes/DelphesClasses.h"')
ROOT.gInterpreter.Declare('#include "external/ExRootAnalysis/ExRootTreeReader.h"')

Welcome to JupyROOT 6.28/06


True

In [6]:
# inputFile = '../eventData/pp2BB1j_atlas_scan_New3/Events/run_04/cdfo_105_100.0_tau_6e+00_delphes_events.root'
# inputFile = '../eventData/pp2BB1j_atlas_scan_New3/Events/run_03/cdfo_106_100.0_tau_6e+00_delphes_events.root'
inputFile = '../eventData/pp2Gluino_atlas/Events/run_10/gluino_400_370.0_tau_3e-01_delphes_events.root'

In [7]:
f = ROOT.TFile(inputFile,'read')
tree = f.Get("Delphes")
nevts = tree.GetEntries()
ntotal = 0
for ievt in range(nevts):    
    
    ntotal += 1
    tree.GetEntry(ievt)   
    weightPB = tree.Weight.At(1).Weight

    metCalo = tree.MissingETCalo.At(0).MET
    llps = getLLPs(tree.bsm,tree.bsmDirectDaughters,tree.bsmFinalDaughters,tree.bsmMothers)
    # hscpCandidates = getHSCPCandidates(tree.isoRhadrons,tree.rhadronDaughters,llps)
    break
# f.Close()

In [8]:
llps

[1000021 (106), 1000021 (106)]

In [9]:
for llp in llps:
    print(llp.PID,llp.E,llp.Charge,llp.getCharge(),llp.SumPtCharged)
    for m in llp.mothers:
        print('  ',m.PID,m.E)

1000021 609.0592041015625 0 0.0
   1009223 610.657958984375
1000021 1205.9241943359375 0 1.0
   -1009213 1209.0897216796875


In [10]:
for p in tree.isoLLPs:
    print(p.PID,p.Mass,p.Status,p.E,p.D1,p.D2)

1000021 400.0 106 609.0592041015625 0 2
1000021 400.0 106 1205.9241943359375 3 5


In [6]:
for ip in range(tree.bsmDirectDaughters.GetEntries()):
    p = tree.bsmDirectDaughters.At(ip)
    print(p.PID,p.Mass,p.Status,p.Charge)

-1 0.33000001311302185 23 0
1 0.33000001311302185 23 0
1000022 370.0 1 0
-1 0.33000001311302185 23 0
1 0.33000001311302185 23 0
1000022 370.0 1 0


In [14]:
for p in tree.bsm:
    # p = tree.bsm.At(ip)
    print(p.PID,p.Mass,p.E,p.Status,p.M1,p.M2,p.D1,p.D2)


for p in tree.isoLLPs:
    # p = tree.bsm.At(ip)
    print(p.PID,p.Mass,p.E,p.Status,p.M1,p.M2,p.D1,p.D2)    

1000021 400.0 609.0592041015625 106 0 0 0 2
1000021 400.0 1205.9241943359375 106 1 1 3 5
1000021 400.0 609.0592041015625 106 0 0 0 2
1000021 400.0 1205.9241943359375 106 1 1 3 5


In [8]:
for ip in range(tree.bsmMothers.GetEntries()):
    p = tree.bsmMothers.At(ip)
    print(p.PID,p.Mass,p.Status,p.Charge)

1009113 401.174560546875 104 -999
-1009213 401.04998779296875 104 -999


In [12]:
for ip,p in enumerate(tree.bsmMothers):
    print(p.PID,p.Mass,p.Status,p.Charge)

1009113 401.174560546875 104 -999
-1009213 401.04998779296875 104 -999


In [9]:
candidates = []    
for ip in range(tree.rhadron.GetEntries()):
    rhadron = tree.rhadron.At(ip)
    print('R-hadron:',rhadron.PID)
    # if abs(rhadron.Charge) == 1: # Skip neutral or multicharged particles
        # continue
    
    # Map rhadron to llp (parton)
    # (check which llp came from the R-hadron)
    rhadron_ddaughters = []
    for jp in range(tree.rhadronDaughters.GetEntries()):
        d = tree.rhadronDaughters.At(jp)
        print('  dE=',d.E)
        for llp in llps:
            print(llp.PID,llp.E,llp is d)
        if d.M1 == ip:
            rhadron_ddaughters.append(d)
    print([(d.PID,d.Charge) for d in rhadron_ddaughters])
    totalCharge = sum([d.Charge for d in rhadron_ddaughters])
    x = rhadron_ddaughters[0].X
    totalE = sum([d.E for d in rhadron_ddaughters])
    print('  Charge=',totalCharge,'X=',x,'totalE = ',totalE,rhadron.E)


R-hadron: 1009113


AttributeError: 'TTree' object has no attribute 'rhadronDaughters'

In [21]:
tree.bsmMothers.GetEntries()

0

In [8]:
candidates

[]

In [8]:
for p in tree.isoLLPs:
    # p = tree.bsmFinalDaughters.At(ip)
    print(p.PID,p.Mass,p.Status,p.Charge)

AttributeError: 'TTree' object has no attribute 'isoLLPs'

In [38]:
for ip in range(tree.Particle.GetEntries()):
    p = tree.Particle.At(ip)
    if abs(p.PID) < 10000:
        continue
    if p.Charge == 0:
        continue
    print(p.PID,p.Status,p.Charge,p.Mass)

2212 4 1 0.9382699728012085
2203 63 1 0.7713299989700317
2212 4 1 0.9382699728012085
-1009002 103 -999 400.70001220703125
-1009002 102 -999 400.70001220703125
1009323 104 -999 401.2250061035156
1009002 103 -999 400.5249938964844
1009002 102 -999 400.5249938964844
1009223 104 -999 401.04998779296875
2203 72 1 0.7713299989700317
213 2 1 0.7540422677993774
-211 1 -1 0.13956999778747559
211 1 1 0.13956999778747559
-213 2 -1 0.754854679107666
211 1 1 0.13956999778747559
-213 2 -1 0.7838952541351318
211 1 1 0.13956999778747559
-211 1 -1 0.13956999778747559
213 2 1 1.2140008211135864
-213 2 -1 0.6929205060005188
2212 1 1 0.9382699728012085
-211 1 -1 0.13956999778747559
321 1 1 0.4936800003051758
-211 1 -1 0.13956999778747559
211 1 1 0.13956999778747559
-321 1 -1 0.4936800003051758
211 1 1 0.13956999778747559
211 1 1 0.13956999778747559
-211 1 -1 0.13956999778747559
213 2 1 0.8911933302879333
-211 1 -1 0.13956999778747559
211 1 1 0.13956999778747559
-211 1 -1 0.13956999778747559
321 1 1 0.4936